In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 

2023-04-25 14:19:23.491187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 14:19:24.412405: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-25 14:19:24.412465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-25 14:19:24.412473: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 

In [ ]:
!wget --no-check-certificate \
    https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

In [3]:
data = open('robert_frost.txt').read()
corpus = data.lower().split("\n")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [4]:
model = Sequential()
model.add(Embedding(total_words, 128, input_length=max_sequence_len - 1))
model.add(Bidirectional(LSTM(120, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(96))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))


model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

print(model.summary())

2023-04-25 14:19:34.706367: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-04-25 14:19:34.706492: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: candyland
2023-04-25 14:19:34.706515: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: candyland
2023-04-25 14:19:34.706809: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.60.11
2023-04-25 14:19:34.706879: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.60.11
2023-04-25 14:19:34.706890: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.60.11
2023-04-25 14:19:34.707555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with on

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 128)           290944    
                                                                 
 bidirectional (Bidirectiona  (None, 11, 240)          239040    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 11, 240)           0         
                                                                 
 lstm_1 (LSTM)               (None, 96)                129408    
                                                                 
 dense (Dense)               (None, 1136)              110192    
                                                                 
 dense_1 (Dense)             (None, 2273)              2584401   
                                                        

In [5]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
298/298 [==============================] - 25s 69ms/step - loss: 6.8039 - accuracy: 0.0471
Epoch 2/100
298/298 [==============================] - 17s 56ms/step - loss: 6.2517 - accuracy: 0.0488
Epoch 3/100
298/298 [==============================] - 14s 46ms/step - loss: 6.0904 - accuracy: 0.0512
Epoch 4/100
298/298 [==============================] - 22s 75ms/step - loss: 5.9859 - accuracy: 0.0563
Epoch 5/100
298/298 [==============================] - 19s 65ms/step - loss: 5.8914 - accuracy: 0.0589
Epoch 6/100
298/298 [==============================] - 20s 66ms/step - loss: 5.7801 - accuracy: 0.0637
Epoch 7/100
298/298 [==============================] - 19s 65ms/step - loss: 5.6745 - accuracy: 0.0679
Epoch 8/100
298/298 [==============================] - 20s 67ms/step - loss: 5.5861 - accuracy: 0.0698
Epoch 9/100
298/298 [==============================] - 14s 46ms/step - loss: 5.5156 - accuracy: 0.0718
Epoch 10/100
298/298 [==============================] - 13s 44ms/step - l

In [26]:
seed_text = "The man who taught ML"
next_words = 50
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list,verbose=0), axis=-1)
	output_word = ""
	# print(predicted)
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word

print(seed_text)

The man who taught ML had no quarrel from forty years ago as ' ' ' ' one more ' to having each step who clasps them are toffile right ' ' ' of family ' i us toffile to care ' ' the night ' ' ' his father glacier round as that slippery
